# Guide Calling Using the Cellranger Module HDF5 File 
## Utilising Direct Capture Perturb-Seq to Deciphering Alternative Promoter Usage  

1. Run through the 10x Cellranger pipeline and velocyto for single cell RNAseq quatification and using (2) guides quantifiction. all found in the cellranger files folder **bash**
2.  Guide Calling for dual guide. Use repogle method to take molecule.h5 generated by cellranger and py to run through repogle version of guide calling or use cellranger_guidecalling.ipynb for Direct Capture Perturb-Seq dual guide. Formed guide-specific lists of cells.
3. Pseudobulk analysis.
    A. Seperation of guide-specific fastq files. **bash**
    B. Whippet pseudobulk for transcript specific analysis, post UMI deduplication.  **bash**
    C. Transcript quality control. **R**
    D. Whippet result visualisation.
4. Normalisation of adata object and E-distance of KD
5. Check gene and neighboring gene expression
6. Create individual umaps per gene of interest
    A. UMAPs 
    B. Rand Index score
7. Cell phase assignment model from FUCCI-matched single cell paper (GSE146773)
8. **Differential Expression analysis.**
    A. Find the shared P1 and P2 genes. 
    B. Check the shared P1 and P2 across different protospacers with the same A/B and C/D.
9.  CNV Score & Numbat to quantify and Velocity quantification with loom file
10. ESR1-specific analysis from proliferation analysis to rt-qpcr
11.  Spectra analysis and visualisation for pathway enrichment

In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2
#general
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import hdf5plugin

#form a location
loc="/Users/helenking/Desktop/Weatheritt_Lab_Y2/alt-prom-crispr-fiveprime/"

import seaborn as sns
from tqdm.notebook import tqdm
import scperturb
import sys
sys.path.append(loc+'scripts/')
from apu_analysis import *
import scperturb
import infercnvpy as cnv
from apu_analysis.cell_import import CellPopulation
from IPython.display import clear_output
pd.options.display.float_format = '{:.4f}'.format
import matplotlib.pyplot as plt

#for this python
from scipy.special import rel_entr
import sklearn.cluster as cluster
import umap
from scipy import stats
from scipy.stats import bootstrap
import statsmodels.api as sm
from statsmodels.stats.multitest import multipletests
from numpy import reshape
from numpy import array
from sklearn.decomposition import PCA
import scipy.stats as stats
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from scipy.stats import mannwhitneyu

# Taken from: 
# Adamson, B.A., Norman, T.M., *et al.* "A multiplexed CRISPR screening platform enables systematic dissection of the unfolded protein response", *Cell*, 2016.
# My experiment deals with two KDs- one of the MP, one of the AP using two guides. Positve controls include GINS1 ect. This is a combnatorial KD double for the same gene. No treatments were used

# colours using garvan 
color1 ='#4d00c7'
palecolor1="#b366ff"
color2= '#da3c07'
palecolor2="#ff8954"
color3='#05d3d3'
color4='#c6c7c5'
color4="#434541"
color5="#eb31e1"
color6="#3175eb"
color7="#a7eb31"
color8="#b366ff"
color9="#ff8954"
color10="#35c9d4"

# #use viridis
# color1="#fde725"
# color2="#7ad151"
# color3="#22a884"
# color4="#2a788e"
# color5="#2a788e"
# color6='#440154'
# %%capture


# Create the color palette
palette = sns.color_palette([palecolor1,palecolor2,color3,color4])
palette2 = sns.color_palette([color1, color2, palecolor1, palecolor2,color5,color6 ,color7])

# # Create the color palette
# palette = sns.color_palette([color1, color2,color3])
new_palette = sns.color_palette([color1, color2,color1, color2,color1, color2,color1, color2,color1, color2,color1, color2, color3, color4])

from random import randrange
pvalue=0.05

print("Scanpy", sc.__version__)
%matplotlib inline

/Users/helenking/anaconda3/envs/apu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Scanpy 1.10.3


In [2]:
%%capture
adata = ad.read_h5ad(loc+"files/adata_normalised.h5ad")
#set the directio
#perform log1p on the data
adata.obs["guide_assignment_simple"]=np.where(((adata.obs["guide_assignment"].str.startswith("non"))|(adata.obs["guide_assignment"].str.startswith("Non"))), "non-targeting", adata.obs["guide_assignment"])
gene_list_select=adata.obs["guide_assignment_simple"][adata.obs["guide_assignment_simple"]!="non-targeting"].drop_duplicates().tolist()
%%capture
# sc.tl.rank_genes_groups(adata, 'guide_assignment_simple', groups=gene_list_select,reference="non-targeting", method='wilcoxon', use_raw=False)
sc.tl.rank_genes_groups(adata, 'guide_assignment_simple', groups=gene_list_select,reference="non-targeting", method='t-test', use_raw=False)

In [5]:
#get the differentially expressed genes
# %%capture 
gene_list=[]
pvalue=0.05
number=1000
sortby="pvals_adj_1"
min_fc=1
#get the highly_variable genes
extract_highly_variable_genes=adata.var[adata.var["highly_variable"]==True].index
gene_list_recursive=adata.obs["perturbation"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values
#remove ARNT and if gene_A CSNK1E

In [6]:
#get the differentially expressed genes
# %%capture 
gene_list=[]
pvalue=0.05
logFC=None
number=1000
sortby="rank_1"
min_fc=0.5
#get the highly_variable genes
extract_highly_variable_genes=adata.var[adata.var["highly_variable"]==True].index
gene_list_recursive=adata.obs["perturbation"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values
#remove ARNT and if gene_A CSNK1E
gene_list_recursive=gene_list_recursive[~gene_list_recursive.isin(["ARNT","CSNK1E"])]
for gene in adata.obs["perturbation"][(adata.obs["guide_id"]!="non-targeting_Control") & (adata.obs["successfulKD"]=="True")].drop_duplicates().values:
    # print(gene)
    #AP
    gene_AP=gene+"_AP"
    if gene_AP not in adata.obs["guide_id"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values:
        continue
    elif gene in ["ARNT"]:
        print("ARNT")
    else:
        for promoter in ["MP","AP"]:
            gene_promoter=gene+"_"+promoter+"_"+str(1)
            gene_promoter_2=gene+"_"+promoter+"_"+str(2)
            gene_promoter_random_1=gene_list_recursive[randrange(len(gene_list_recursive))]+"_"+promoter+"_"+str(1)
            gene_promoter_random_2=gene_list_recursive[randrange(len(gene_list_recursive))]+"_"+promoter+"_"+str(2)
            ##form a dataframe with custom colummns 
            #subset random gene from gene list
            protospacer_1= sc.get.rank_genes_groups_df(adata, group=gene_promoter,pval_cutoff=pvalue)
            #order by the absolute value of the logfoldchange
            protospacer_1["abs_logfoldchanges"]=protospacer_1["logfoldchanges"].abs()
            protospacer_1.sort_values(by="abs_logfoldchanges",ascending=False,inplace=True)
            #add a rank column
            protospacer_1["rank"]=range(1,len(protospacer_1)+1)
            protospacer_2= sc.get.rank_genes_groups_df(adata, group=gene_promoter_2,pval_cutoff=pvalue)
            protospacer_2["abs_logfoldchanges"]=protospacer_2["logfoldchanges"].abs()
            protospacer_2.sort_values(by="abs_logfoldchanges",ascending=False,inplace=True)
            #add a rank column
            protospacer_2["rank"]=range(1,len(protospacer_2)+1)
            protospacer_1_random= sc.get.rank_genes_groups_df(adata, group=gene_promoter_random_1,pval_cutoff=pvalue)
            protospacer_1_random["abs_logfoldchanges"]=protospacer_1_random["logfoldchanges"].abs()
            protospacer_1_random.sort_values(by="abs_logfoldchanges",ascending=False,inplace=True)
            #add a rank column
            protospacer_1_random["rank"]=range(1,len(protospacer_1_random)+1)
            protospacer_2_random= sc.get.rank_genes_groups_df(adata, group=gene_promoter_random_2,pval_cutoff=pvalue)
            protospacer_2_random["abs_logfoldchanges"]=protospacer_2_random["logfoldchanges"].abs()
            protospacer_2_random.sort_values(by="abs_logfoldchanges",ascending=False,inplace=True)
            #add a rank column
            protospacer_2_random["rank"]=range(1,len(protospacer_2_random)+1)
            gene_df=protospacer_1.merge(protospacer_2, on="names", how="left", suffixes=('_1', '_2')).dropna()
            #filter for highly variable genes
            gene_df_random_1=protospacer_1.merge(protospacer_1_random, on="names", how="left", suffixes=('_1', '_2')).dropna()
            gene_df_random_2=protospacer_1.merge(protospacer_2_random, on="names", how="left", suffixes=('_1', '_2')).dropna()
            rank_columns=["rank_1","rank_2"]
            #get the correlation for  scores_1 and scores_2 for corr
            correlation_1=gene_df.sort_values(by=sortby, ascending=True)[rank_columns].corr(method='spearman',numeric_only=True)
            correlation_random_1=gene_df_random_1.sort_values(by=sortby, ascending=True)[rank_columns].corr(method='spearman',numeric_only=True)
            correlation_random_2=gene_df_random_2.sort_values(by=sortby, ascending=True)[rank_columns].corr(method='spearman',numeric_only=True)
            #create a row wtht he gene name 
            gene_list_row=[gene,promoter]
            gene_list_row.extend([correlation_1.loc["rank_1","rank_2"],correlation_random_1.loc["rank_1","rank_2"],correlation_random_2.loc["rank_1","rank_2"]])
            gene_list.append(gene_list_row)
            

#flatten list of lists into dataframe
gene_df_final=pd.DataFrame(gene_list, columns=["gene","promoter","scores_1_targeted","scores_1_random","scores_2_random"])     


In [7]:
#get the differentially expressed genes
# %%capture 
# pvalue=0.01
gene_list=[]
gene_list_recursive=adata.obs["perturbation"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values
#remove ARNT and if gene_A CSNK1E
gene_list_recursive=gene_list_recursive[~gene_list_recursive.isin(["ARNT","CSNK1E"])]
for gene in adata.obs["perturbation"][(adata.obs["guide_id"]!="non-targeting_Control") & (adata.obs["successfulKD"]=="True")].drop_duplicates().values:
    print(gene)
    #AP
    gene_AP=gene+"_AP"
    if gene_AP not in adata.obs["guide_id"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values:
        continue
    elif gene in ["ARNT"]:
        print("ARNT")
    else:
        for promoter in ["MP","AP"]:
            gene_promoter=gene+"_"+promoter+"_"+str(1)
            protospacer_1= sc.get.rank_genes_groups_df(adata, group=gene_promoter,pval_cutoff=pvalue)
            protospacer_1=protospacer_1['names'].values
            
            gene_promoter=gene+"_"+promoter+"_"+str(2)
            protospacer_2= sc.get.rank_genes_groups_df(adata, group=gene_promoter,pval_cutoff=pvalue)
            protospacer_2=protospacer_2['names'].values
            
            gene_promoter=gene_list_recursive[randrange(len(gene_list_recursive))]+"_"+promoter+"_"+str(1)
            protospacer_1_random= sc.get.rank_genes_groups_df(adata, group=gene_promoter,pval_cutoff=pvalue)
            protospacer_1_random=protospacer_1_random["names"].values

            gene_promoter=gene_list_recursive[randrange(len(gene_list_recursive))]+"_"+promoter+"_"+str(2)
            protospacer_2_random= sc.get.rank_genes_groups_df(adata, group=gene_promoter,pval_cutoff=pvalue)
            protospacer_2_random=protospacer_2_random["names"].values
            
            
            overlap = [x for x in protospacer_1 if x in protospacer_2]
            overlap_random_1 = [x for x in overlap if x in protospacer_1_random]
            overlap_random_2 = [x for x in overlap if x in protospacer_2_random]
            #if overlap protospacer_1 protospacer_2 overlap_random_2 overlap_random_1 = 0 set to 1
            # print(protospacer_1,protospacer_2,overlap,overlap_random_1,overlap_random_2)
            try: 
                overlap_1=len(overlap)/len(protospacer_1)
                overlap_2=len(overlap)/len(protospacer_2)
                overlap_random_1=len(overlap_random_1)/len(protospacer_1_random)
                overlap_random_2=len(overlap_random_2)/len(protospacer_2_random)
            except ZeroDivisionError:
                overlap_1=0
                overlap_2=0
                overlap_random_1=0
                overlap_random_2=0
            
            gene_df=pd.DataFrame([[gene,promoter,overlap_1,overlap_2,overlap_random_1,overlap_random_2]])
            # gene_df_details=pd.DataFrame([[gene,promoter,protospacer_1,protospacer_2,overlap,len(protospacer_1),len(protospacer_2),len(overlap)]])

            # print(gene_df.shape)
            gene_list.append(gene_df)
            # break


# col_names=["Gene","Promoter","Protospacer_1","Protospacer_2","Overlap","Protospacer_Random_1","Protospacer_Random_2"]
#add col_names
col_names=["Gene","Promoter","Overlap_1","Overlap_2","Overlap_Random_1","Overlap_Random_2"]
all=pd.DataFrame(pd.concat(gene_list))
all.columns=col_names


AHCYL1
SP1
TGIF1
YWHAZ
P4HB
NCOR2
JARID2
ESR1
PA2G4
GPBP1
DHX30
SAFB
NFE2L2
CHD8
SIN3A
PSMC5
RC3H2
ZNF3
GTF2F1
ADAR
NR2F2
BZW1
FHL2
SET
MYBBP1A
CUX1
NBN
CALR
APP
STAG2
PKM
BRIP1
GATA3
RBM47


In [9]:
#get the tests between two unpaired parametric groups
#melt for random
all_melt=all.melt(id_vars=["Gene","Promoter"])
all.index=all["Gene"]+"_"+all["Promoter"]   
print(stats.ttest_ind(all["Overlap_1"],all["Overlap_2"],equal_var=False))
print(stats.ttest_ind(all["Overlap_Random_1"],all["Overlap_Random_2"],equal_var=False))
print(stats.ttest_ind(all["Overlap_1"],all["Overlap_Random_1"],equal_var=False))
print(stats.ttest_ind(all["Overlap_2"],all["Overlap_Random_2"],equal_var=False))



TtestResult(statistic=-1.200758436011414, pvalue=0.23196364293947955, df=133.97197243239725)
TtestResult(statistic=0.14299988815177855, pvalue=0.8865064991701896, df=133.1170801399246)
TtestResult(statistic=10.31768853968277, pvalue=6.501330271631454e-18, df=111.19538835724522)
TtestResult(statistic=11.997174432050567, pvalue=2.1194060394826254e-21, df=104.97349430144247)


In [ ]:
#plot the overlap between the two guides
sns.set(style="white")
sns.set_context("talk")
fig, ax = plt.subplots(figsize=(8, 5))
new_pallete=sns.color_palette(["#B4D88C","#C67B6A","#53449B","#8C1F1C"])
#show the percentage over "Overlap_length"/"Protospacer_1_length"
sns.kdeplot(data=all_melt,x="value",hue="variable",palette=new_pallete,  
)
#save plot
plt.savefig(loc+"figures/overlap_gene_signature.pdf")
gene_list_select=adata.obs["guide_id"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values
sc.tl.rank_genes_groups(adata, 'guide_id', groups=gene_list_select.to_list(),reference="non-targeting_Control", method='t-test')
genelist=adata.obs['guide_id'].drop_duplicates().tolist()

In [14]:
%%capture
sc.tl.rank_genes_groups(adata, 'guide_id',groups=genelist, reference="non-targeting_Control", use_raw=False)

#get target gene logfoldchanges
#get the differentially expressed genes 
gene_list=[]
comp_gene_list=[]

for gene in adata.obs["perturbation"][(adata.obs["guide_id"]!="non-targeting_Control")& (adata.obs["successfulKD"]=="True")].drop_duplicates().values:
    # print(gene)
    #AP
    gene_AP=gene+"_AP"
    gene_MP=gene+"_MP"

    if gene_AP not in adata.obs["guide_id"][(adata.obs["guide_id"]!="non-targeting_Control")& (adata.obs["successfulKD"]=="True")].drop_duplicates().values:
        continue
    else:
        ##form a dataframe with custom colummns 
        MP = sc.get.rank_genes_groups_df(adata, group=gene_MP)
        MP=MP[(MP["names"]==gene)]
        AP = sc.get.rank_genes_groups_df(adata, group=gene_AP)        
        AP=AP[(AP["names"]==gene)]
        promoter_line=[gene,MP["scores"].values[0],AP["scores"].values[0],MP["pvals_adj"].values[0],AP["pvals_adj"].values[0]]
        comp_gene_list.append(promoter_line)
        # break
#convert to dataframe
comp_gene_list=pd.DataFrame(comp_gene_list, columns=["gene","MP_logfoldchange","AP_logfoldchange","MP_pvalue","AP_pvalue"])


In [16]:
%%capture
gene_list=[]
comp_gene_list=[]

for gene in adata.obs["perturbation"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values:
    print(gene)
    #AP
    gene_AP=gene+"_AP"
    gene_MP=gene+"_MP"

    if gene_AP not in adata.obs["guide_id"][adata.obs["guide_id"]!="non-targeting_Control"].drop_duplicates().values:
        continue
    else:
        ##form a dataframe with custom colummns 
        MP = sc.get.rank_genes_groups_df(adata, group=gene_MP, pval_cutoff=0.05)
        # MP = MP[MP["logfoldchanges"].abs()>0.1]
        MP_names=MP['names'].values
        # ,   pval_cutoff=0.05)['names']
        # sc.tl.rank_genes_groups(adata_ideal_subset, 'guide_identity', groups=['MYBBP1A_AP'], reference='Negative_Control_Control', method='wilcoxon')
        AP = sc.get.rank_genes_groups_df(adata, group=gene_AP, pval_cutoff=0.05)
        # AP = AP[AP["logfoldchanges"].abs()>0.5]

        AP_names=AP['names'].values

        # , pval_cutoff=0.05, log2fc_min=0)['names']
        overlap = [x for x in MP_names if x in AP_names]
        #get average log2fc for MP, AP and overlap
        
        gene_df=pd.DataFrame([[gene,MP_names,AP_names,overlap,len(MP_names),len(AP_names),len(overlap)]])
        print(gene_df.shape)
        gene_list.append(gene_df)


        ##add to another df which includes the gene and their direction
        MP["Guide"]=gene_MP
        comp_gene_list.append(MP)
        AP["Guide"]=gene_AP
        comp_gene_list.append(AP)
        # break

col_names=["Target_Gene","MP_Genes","AP_Genes","Overlap_Genes","MP_Gene_Num","AP_Gene_Num","Overlap_Gene_Num"]
all=pd.DataFrame(pd.concat(gene_list))
all.columns=col_names

#only filter for genes with successfulKD 
all_filt=all[all["Target_Gene"].isin(adata.obs["gene"][adata.obs["successfulKD"]=="True"].drop_duplicates().values)]
all_filt.to_csv(loc+"files/singlecell_shortread_analysis/differential_exp_list.csv")


In [17]:
all_comp=pd.DataFrame(pd.concat(comp_gene_list))
all_comp.columns=MP.columns
all_comp["Promoter"]=all_comp["Guide"].str.split("_").str.get(1)
all_comp["Target_Gene"]=all_comp["Guide"].str.split("_").str.get(0)
all_comp_mean=all_comp.groupby("Target_Gene").agg({'logfoldchanges': 'mean'})
all_comp_mean.reset_index(inplace=True)
all_comp_mean.columns=["Target_Gene","logfoldchanges_mean"]
all_comp=all_comp.merge(all_comp_mean,on="Target_Gene")
all_comp_mean=all_comp.groupby("Target_Gene").agg({'Target_Gene':"count"}).rename(columns={"Target_Gene":"Count"})
all_comp_mean.reset_index(inplace=True)
all_comp=all_comp.merge(all_comp_mean,on="Target_Gene")
all_comp=all_comp.sort_values(by="Guide")
all_comp.to_csv(loc+"files/singlecell_shortread_analysis/differential_exp_list_full.csv")

In [18]:
all["Mutual_Exclusivity"]=all["Overlap_Gene_Num"]/(all["MP_Gene_Num"]+all["AP_Gene_Num"])

n_term=pd.read_table("alt-prom-crispr-fiveprime/reference/all_pivot_simple_nterm.txt")
n_term=all.merge(n_term, right_on="Gene_symbol",left_on="Target_Gene")

n_term=n_term[["MP_Gene_Num","AP_Gene_Num","Overlap_Gene_Num","Mutual_Exclusivity","Ensembl_ID","Gene_symbol","Nterminus_Change"]]
# all_comp_pivot
all_comp_all=all_comp.merge(n_term, left_on="Target_Gene", right_on="Gene_symbol" ,how="outer")
all_comp_all=all_comp_all.sort_values("Nterminus_Change",ascending=True)